# DES Y6 Deep Field Exposures:  Photometric Zeropoints tied to SDSS DR13

## 1. Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import glob
import math
import os
import subprocess
import sys
import gc

import glob
import pickle

import easyaccess as ea

#import AlasBabylon

import fitsio
from astropy.io import fits
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, vstack

import tempfile

import matplotlib.pyplot as plt
 
%matplotlib inline

In [2]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. User Input

### 2.1.  General User Input  

In [3]:
verbose = 1
tag_des = 'Y6A2_FINALCUT'      # Official tag for DES Y6A2_FINALCUT
tag_decade = 'DECADE_FINALCUT' # Tag for DECADE
rawdata_dir = '../RawData'
zeropoints_dir='../Zeropoints'

#bandList = ['u', 'g', 'r', 'i', 'z']   # Oops!  Missing transformation eqn for Y-band...
bandList = ['u', 'g', 'r', 'i', 'z']

### 2.2. Logical Variables to Indicate which Code Cells to Run

In [4]:
do_calc_sdss_zps = True

### 2.3. Sky Region Definitions

In [5]:
region_name_list = [
                    'VVDSF14', 
                    'VVDSF22', 
                    'DEEP2', 
                    'SN-E', 
                    'SN-X_err', 
                    'SN-X', 
                    'ALHAMBRA2', 
                    'SN-S',
                    'SN-C', 
                    'EDFS',
                    'MACS0416',
                    'SN-S_err',
                    'COSMOS'
                    ]

region_ramin = {
                'VVDSF14':208.,
                'VVDSF22':333.,
                'DEEP2':351., 
                'SN-E':6., 
                'SN-X_err':13., 
                'SN-X':32., 
                'ALHAMBRA2':35., 
                'SN-S':39.5,
                'SN-C':50., 
                'EDFS':55.,
                'MACS0416':62.,
                'SN-S_err':83.5,
                'COSMOS':148.
               }

region_ramax = {
                'VVDSF14':212.,
                'VVDSF22':337.,
                'DEEP2':354., 
                'SN-E':12., 
                'SN-X_err':17., 
                'SN-X':38., 
                'ALHAMBRA2':39., 
                'SN-S':44.5,
                'SN-C':56., 
                'EDFS':67.,
                'MACS0416':66.,
                'SN-S_err':88.,
                'COSMOS':153.
               }

region_decmin = {
                'VVDSF14':3.,
                'VVDSF22':-1.5,
                'DEEP2':-1.5, 
                'SN-E':-46., 
                'SN-X_err':-32., 
                'SN-X':-8., 
                'ALHAMBRA2':-0.5, 
                'SN-S':-2.5,
                'SN-C':-31., 
                'EDFS':-51.,
                'MACS0416':-25.5,
                'SN-S_err':-38.,
                'COSMOS':0.5
               }


region_decmax = {
                'VVDSF14':7.,
                'VVDSF22':1.5,
                'DEEP2':1.5, 
                'SN-E':-41., 
                'SN-X_err':-29., 
                'SN-X':-3., 
                'ALHAMBRA2':2.5, 
                'SN-S':1.5,
                'SN-C':-25., 
                'EDFS':-46.,
                'MACS0416':-22.5,
                'SN-S_err':-35.,
                'COSMOS':4.0
               }


for regionName in region_name_list:
    print regionName, region_ramin[regionName], region_ramax[regionName], region_decmin[regionName], region_decmax[regionName]

VVDSF14 208.0 212.0 3.0 7.0
VVDSF22 333.0 337.0 -1.5 1.5
DEEP2 351.0 354.0 -1.5 1.5
SN-E 6.0 12.0 -46.0 -41.0
SN-X_err 13.0 17.0 -32.0 -29.0
SN-X 32.0 38.0 -8.0 -3.0
ALHAMBRA2 35.0 39.0 -0.5 2.5
SN-S 39.5 44.5 -2.5 1.5
SN-C 50.0 56.0 -31.0 -25.0
EDFS 55.0 67.0 -51.0 -46.0
MACS0416 62.0 66.0 -25.5 -22.5
SN-S_err 83.5 88.0 -38.0 -35.0
COSMOS 148.0 153.0 0.5 4.0


### 2.4. Check on Location of TMPDIR...

In [6]:
# Check on TMPDIR... 
tempfile.gettempdir()

'/data/des40.a/data/dtucker/TmpDir'

In [7]:
# Set tmpdir variable to $TMPDIR (for future reference)...
tmpdir = os.environ['TMPDIR']

### 2.5. Create Main Zeropoints Directory (if it does not already exist)...

In [8]:
# Create main Zeropoints directory, if it does not already exist...
if not os.path.exists(zeropoints_dir):
    os.makedirs(zeropoints_dir)

## 3. Useful Modules

In [9]:
def DECam_tie_to_sdss(inputFile, outputFile, band, fluxObsColName, fluxerrObsColName, aggFieldColName, verbose):

    import numpy as np 
    import os
    import sys
    import datetime
    import pandas as pd
    from astropy.table import Table, vstack
    
    #validBandsList = ['u', 'g', 'r', 'i', 'z', 'Y']  # Oops!  Missing transform eqn for Y-band!
    validBandsList = ['u', 'g', 'r', 'i', 'z']

    if band not in validBandsList:
        print """Filter band %s is not currently handled...  Exiting now!""" % (band)
        return 1
    
    reqColList = ['psfMag_u','psfMagErr_u', 
                  'psfMag_g','psfMagErr_g',
                  'psfMag_r','psfMagErr_r',
                  'psfMag_i','psfMagErr_i',
                  'psfMag_z','psfMagErr_z',
                  fluxObsColName,fluxerrObsColName,aggFieldColName]        
      
    # Does the input file exist?
    if os.path.isfile(inputFile)==False:
        print """DECam_tie_to_sdss input file %s does not exist.  Exiting...""" % (inputFile)
        return 1

    # Read inputFile into a pandas DataFrame...
    print datetime.datetime.now()
    print """Reading in %s as a pandas DataFrame...""" % (inputFile)
    t = Table.read(inputFile)
    dataFrame = t.to_pandas()
    print datetime.datetime.now()
    print

    reqColFlag = 0
    colList = dataFrame.columns.tolist()
    for reqCol in reqColList:
        if reqCol not in colList:
            print """ERROR:  Required column %s is not in the header""" % (reqCol)
            reqColFlag = 1
    if reqColFlag == 1:
        print """Missing required columns in header of %s...  Exiting now!""" (inputFile)
        return 1

    # Identify column of the standard magnitude for the given band...
    magStdColName = """MAG_STD_%s""" % (band.upper())
    magerrStdColName = """MAGERR_STD_%s""" % (band.upper())
    
    # Transform SDSS mags to DES mags for the given band...
    dataFrame = transSDSStoDES(dataFrame, band, magStdColName, magerrStdColName)

    # Add a 'MAG_OBS' column and a 'MAG_DIFF' column to the pandas DataFrame...
    dataFrame['MAG_OBS'] = -2.5*np.log10(dataFrame[fluxObsColName])
    dataFrame['MAG_DIFF'] = dataFrame[magStdColName]-dataFrame['MAG_OBS']


    ###############################################
    # Aggregate by aggFieldColName
    ###############################################

    # Make a copy of original dataFrame...
    df = dataFrame.copy()

    # Create an initial mask...
    mask1 = ( (df[magStdColName] >= 0.) & (df[magStdColName] <= 25.) )
    mask1 = ( mask1 & (df[fluxObsColName] > 10.) & (df['FLAGS'] < 2) & (np.abs(df['SPREAD_MODEL']) < 0.01))
    if magerrStdColName != 'None':  
        mask1 = ( mask1 & (df[magerrStdColName] < 0.1) )
    magDiffGlobalMedian = df[mask1]['MAG_DIFF'].median()
    magDiffMin = magDiffGlobalMedian - 5.0
    magDiffMax = magDiffGlobalMedian + 5.0
    mask2 = ( (df['MAG_DIFF'] > magDiffMin) & (df['MAG_DIFF'] < magDiffMax) )
    mask = mask1 & mask2

    # Iterate over the copy of dataFrame 3 times, removing outliers...
    #  We are using "Method 2/Group by item" from
    #  http://nbviewer.jupyter.org/urls/bitbucket.org/hrojas/learn-pandas/raw/master/lessons/07%20-%20Lesson.ipynb
    print "Sigma-clipping..."
    niter = 0
    for i in range(3):

        niter = i + 1
        print """   iter%d...""" % ( niter )

        # make a copy of original df, and then delete the old one...
        newdf = df[mask].copy()
        del df
        # group by aggFieldColName...
        grpnewdf = newdf.groupby([aggFieldColName])
        
        # add/update new columns to newdf
        print datetime.datetime.now()
        newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 3.00*x.std() )
        #newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 2.00*x.std() )
        print datetime.datetime.now()
        del grpnewdf
        print datetime.datetime.now()
        #print newdf

        nrows = newdf['MAG_DIFF'].size
        print """  Number of rows remaining:  %d""" % ( nrows )

        df = newdf
        mask = ( df['Outlier'] == False )  


    # Perform pandas grouping/aggregating functions on sigma-clipped Data Frame...
    print datetime.datetime.now()
    print 'Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...'
    groupedDataFrame = df.groupby([aggFieldColName])
    magZeroMedian = groupedDataFrame['MAG_DIFF'].median()
    magZeroMean = groupedDataFrame['MAG_DIFF'].mean()
    magZeroStd = groupedDataFrame['MAG_DIFF'].std()
    magZeroNum = groupedDataFrame['MAG_DIFF'].count()
    magZeroErr = magZeroStd/np.sqrt(magZeroNum-1)
    print datetime.datetime.now()
    print

    # Rename these pandas series...
    magZeroMedian.name = 'MAG_ZERO_MEDIAN'
    magZeroMean.name = 'MAG_ZERO_MEAN'
    magZeroStd.name = 'MAG_ZERO_STD'
    magZeroNum.name = 'MAG_ZERO_NUM'
    magZeroErr.name = 'MAG_ZERO_MEAN_ERR'

    # Also, calculate group medians for all columns in df that have a numerical data type...
    numericalColList = df.select_dtypes(include=[np.number]).columns.tolist()
    groupedDataMedian = {}
    for numericalCol in numericalColList:
        groupedDataMedian[numericalCol] = groupedDataFrame[numericalCol].median()
        groupedDataMedian[numericalCol].name = """%s_MEDIAN""" % (numericalCol)

    # Create new data frame containing all the relevant aggregate quantities
    #newDataFrame = pd.concat( [magZeroMedian, magZeroMean, magZeroStd, \
    #                           magZeroErr, magZeroNum], \
    #                           join='outer', axis=1 )
    seriesList = []
    for numericalCol in numericalColList:
        seriesList.append(groupedDataMedian[numericalCol])
    seriesList.extend([magZeroMedian, magZeroMean, magZeroStd, \
                               magZeroErr, magZeroNum])
    #print seriesList
    newDataFrame = pd.concat( seriesList, join='outer', axis=1 )
                               

    #newDataFrame.index.rename('FILENAME', inplace=True)

    # Saving catname-based results to output files...
    print datetime.datetime.now()
    print """Writing %s output file (using pandas to_csv method)...""" % (outputFile)
    newDataFrame.to_csv(outputFile, float_format='%.4f')
    print datetime.datetime.now()
    print

    return 0

In [12]:
# Transform SDSS mags into DES mags for this filter band...
def transSDSStoDES(dataFrame, band, magStdColName, magerrStdColName):

    import numpy as np 
    import pandas as pd
    from collections import OrderedDict as odict

    # Transformation coefficients (updated based on fit to DES).
    # mag_des = mag_sdss + A[mag][0]*color_sdss + A[mag][1]
    # (based on A. Drlica-Wagner's https://github.com/kadrlica/desqr/blob/master/desqr/calibrate.py).
    # Values come from DES-doc#9420 (u-band) and the DES DR2 paper (g-,r-,i-,z-bands).
    SDSS = odict([
            ('u', [-0.350, +0.466, -0.479, ]), # [+0.2 < (g-r)_sdss <= 1.4]
            ('g', [-0.061, +0.008]), # [-1.0 < (g-i)_sdss <= 3.5]
            ('r', [-0.155, -0.007]), # [-0.4 < (r-i)_sdss <= 2.0]
            ('i', [-0.166, +0.032]), # [-0.4 < (r-i)_sdss <= 2.0]
            ('z', [-0.056, +0.027]), # [-0.4 < (r-i)_sdss <= 2.0]
            ]) 
    
    A = SDSS

    if band is 'u':
        # u_des = u_sdss - 0.350*(g-r)_sdss**2 + 0.466*(g-r)_sdss - 0.479    [+0.2 < (g-r)_sdss <= 1.4]
        dataFrame[magStdColName] = dataFrame['psfMag_u']+\
            A[band][0]*(dataFrame['psfMag_g']-dataFrame['psfMag_r'])*(dataFrame['psfMag_g']-dataFrame['psfMag_r'])+\
            A[band][1]*(dataFrame['psfMag_g']-dataFrame['psfMag_r'])+\
            A[band][2]
        dataFrame[magerrStdColName] = dataFrame['psfMagErr_u']  # temporary
        mask  = ( (dataFrame['psfMag_g']-dataFrame['psfMag_r']) > +0.2)
        mask &= ( (dataFrame['psfMag_g']-dataFrame['psfMag_r']) <= 1.4)
    elif band is 'g':
        # g_des = g_sdss - 0.061*(g-i)_sdss + 0.008   [-1.0 < (g-i)_sdss <= 3.5]
        dataFrame[magStdColName] = dataFrame['psfMag_g']+\
            A[band][0]*(dataFrame['psfMag_g']-dataFrame['psfMag_i'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['psfMagErr_g']  # temporary
        mask  = ( (dataFrame['psfMag_g']-dataFrame['psfMag_i']) > -1.0)
        mask &= ( (dataFrame['psfMag_g']-dataFrame['psfMag_i']) <= 3.5)
    elif band is 'r':
        # r_des = r_sdss - 0.155*(r-i)_sdss - 0.007   [-0.4 < (r-i)_sdss <= 2.0]
        dataFrame[magStdColName] = dataFrame['psfMag_r']+\
            A[band][0]*(dataFrame['psfMag_r']-dataFrame['psfMag_i'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['psfMagErr_r']  # temporary
        mask  = ( (dataFrame['psfMag_r']-dataFrame['psfMag_i']) > -0.4)
        mask &= ( (dataFrame['psfMag_r']-dataFrame['psfMag_i']) <= 2.0)
    elif band is 'i':
        # i_des = i_sdss - 0.166*(r-i)_sdss + 0.032   [-0.4 < (r-i)_sdss <= 2.0]
        dataFrame[magStdColName] = dataFrame['psfMag_i']+\
            A[band][0]*(dataFrame['psfMag_r']-dataFrame['psfMag_i'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['psfMagErr_i']  # temporary
        mask  = ( (dataFrame['psfMag_r']-dataFrame['psfMag_i']) > -0.4)
        mask &= ( (dataFrame['psfMag_r']-dataFrame['psfMag_i']) <= 2.0)
    elif band is 'z':
        # z_des = z_sdss - 0.056*(r-i)_sdss + 0.027   [-0.4 < (r-i)_sdss <= 2.0]
        dataFrame[magStdColName] = dataFrame['psfMag_z']+\
            A[band][0]*(dataFrame['psfMag_r']-dataFrame['psfMag_i'])+A[band][1]
        dataFrame[magerrStdColName] = dataFrame['psfMagErr_z']  # temporary
        mask  = ( (dataFrame['psfMag_r']-dataFrame['psfMag_i']) > -0.4)
        mask &= ( (dataFrame['psfMag_r']-dataFrame['psfMag_i']) <= 2.0)
    else:
        msg = "Unrecognized band: %s "%band
        raise ValueError(msg)

    dataFrame = dataFrame[mask].copy()

    return dataFrame

## 4. Zeropoints by tying to DES-transformed SDSS DR13 Stars

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

Calculate zeropoints region by region...

In [13]:
%%time 

if do_calc_sdss_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.sdss.fits""" % (subdir, regionName, band)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    

                    status = DECam_tie_to_sdss(inputFile, outputFile, 
                                                    band, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.?.u.sdss.fits
../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.sdss.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.VVDSF14.0.g.sdss.csv
2022-01-04 18:27:01.133537
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.sdss.fits as a pandas DataFrame...
2022-01-04 18:27:02.020849

Sigma-clipping...
   iter1...
2022-01-04 18:27:02.119470
2022-01-04 18:27:02.613105
2022-01-04 18:27:02.613454
  Number of rows remaining:  13776
   iter2...
2022-01-04 18:27:02.641850
2022-01-04 18:27:02.957722
2022-01-04 18:27:02.958003
  Number of rows remaining:  13448
   iter3...
2022-01-04 18:27:02.977215
2022-01-04 18:27:03.356285
2022-01-04 18:27:03.356625
  Number of rows remaining:  13237
2022-01-04 18:27:03.357915
Performing grouping/aggregating functions on sigma-clipp

2022-01-04 18:27:12.133860

Sigma-clipping...
   iter1...
2022-01-04 18:27:12.159660
2022-01-04 18:27:12.161634
2022-01-04 18:27:12.161802
  Number of rows remaining:  0
   iter2...
2022-01-04 18:27:12.165372
2022-01-04 18:27:12.167127
2022-01-04 18:27:12.167426
  Number of rows remaining:  0
   iter3...
2022-01-04 18:27:12.169900
2022-01-04 18:27:12.171488
2022-01-04 18:27:12.171673
  Number of rows remaining:  0
2022-01-04 18:27:12.172541
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 18:27:12.175925

2022-01-04 18:27:12.234236
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.u.sdss.csv output file (using pandas to_csv method)...
2022-01-04 18:27:12.235178


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.g.sdss.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.g.sdss.csv
2022-01-04 18:27:12.245036
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.g.sdss.fits as a pandas DataFrame...
2022-01

2022-01-04 18:29:25.288615


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.0.r.sdss.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.0.r.sdss.csv
2022-01-04 18:29:25.710231
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.0.r.sdss.fits as a pandas DataFrame...
2022-01-04 18:29:40.371962

Sigma-clipping...
   iter1...
2022-01-04 18:29:43.883019
2022-01-04 18:30:03.426414
2022-01-04 18:30:03.435233
  Number of rows remaining:  398166
   iter2...
2022-01-04 18:30:04.715703
2022-01-04 18:30:23.965352
2022-01-04 18:30:23.973343
  Number of rows remaining:  386970
   iter3...
2022-01-04 18:30:24.864886
2022-01-04 18:30:43.918365
2022-01-04 18:30:43.925919
  Number of rows remaining:  382704
2022-01-04 18:30:43.928125
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 18:30:44.073681

2022-01-04 18:30:46.150653
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.0.r.sdss.csv output file (using pandas to_csv met

2022-01-04 18:40:13.642020

Sigma-clipping...
   iter1...
2022-01-04 18:40:13.809701
2022-01-04 18:40:14.961837
2022-01-04 18:40:14.962354
  Number of rows remaining:  37570
   iter2...
2022-01-04 18:40:15.031268
2022-01-04 18:40:16.175225
2022-01-04 18:40:16.175745
  Number of rows remaining:  36409
   iter3...
2022-01-04 18:40:16.211838
2022-01-04 18:40:17.325817
2022-01-04 18:40:17.326345
  Number of rows remaining:  35686
2022-01-04 18:40:17.327297
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 18:40:17.341026

2022-01-04 18:40:17.510444
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.g.sdss.csv output file (using pandas to_csv method)...
2022-01-04 18:40:17.853851


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.r.sdss.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.r.sdss.csv
2022-01-04 18:40:17.869999
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.r.sdss.fits 

2022-01-04 18:43:07.012974

Sigma-clipping...
   iter1...
2022-01-04 18:43:13.088538
2022-01-04 18:43:37.924058
2022-01-04 18:43:37.934023
  Number of rows remaining:  531981
   iter2...
2022-01-04 18:43:40.138604
2022-01-04 18:44:04.788687
2022-01-04 18:44:04.798627
  Number of rows remaining:  520013
   iter3...
2022-01-04 18:44:06.191767
2022-01-04 18:44:31.012387
2022-01-04 18:44:31.022690
  Number of rows remaining:  515444
2022-01-04 18:44:31.025055
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 18:44:31.281473

2022-01-04 18:44:34.376640
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-S.0.z.sdss.csv output file (using pandas to_csv method)...
2022-01-04 18:44:41.102227



# # # # # # # # # # # # # # #
Working on region SN-C
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.u.sdss.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.u.sdss.csv
2022-01-04 18:44:41.377048
Reading in ../RawData/Ex

2022-01-04 18:44:45.583519

Sigma-clipping...
   iter1...
2022-01-04 18:44:45.602169
2022-01-04 18:44:45.604185
2022-01-04 18:44:45.604348
  Number of rows remaining:  0
   iter2...
2022-01-04 18:44:45.607946
2022-01-04 18:44:45.609697
2022-01-04 18:44:45.609859
  Number of rows remaining:  0
   iter3...
2022-01-04 18:44:45.612759
2022-01-04 18:44:45.614471
2022-01-04 18:44:45.614630
  Number of rows remaining:  0
2022-01-04 18:44:45.615551
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 18:44:45.618679

2022-01-04 18:44:45.675048
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.3.z.sdss.csv output file (using pandas to_csv method)...
2022-01-04 18:44:45.675751



# # # # # # # # # # # # # # #
Working on region EDFS
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.EDFS.?.u.sdss.fits
No files matching template ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.EDFS

2022-01-04 18:44:51.865141

Sigma-clipping...
   iter1...
2022-01-04 18:44:52.217473
2022-01-04 18:44:54.709591
2022-01-04 18:44:54.710504
  Number of rows remaining:  73305
   iter2...
2022-01-04 18:44:54.793668
2022-01-04 18:44:57.266959
2022-01-04 18:44:57.268206
  Number of rows remaining:  72164
   iter3...
2022-01-04 18:44:57.345544
2022-01-04 18:44:59.836394
2022-01-04 18:44:59.837375
  Number of rows remaining:  71790
2022-01-04 18:44:59.838422
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 18:44:59.862792

2022-01-04 18:45:00.182376
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.u.sdss.csv output file (using pandas to_csv method)...
2022-01-04 18:45:00.868382


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.g.sdss.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.g.sdss.csv
2022-01-04 18:45:00.890536
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.g.sdss.fits as a pandas 

Combine region-by-region results into a single file...

In [14]:
%%time 

if do_calc_sdss_zps: 
        
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.sdss.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.sdss.csv""" % (subdir, band)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.sdss.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band u
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.u.sdss.csv

# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.g.sdss.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.r.sdss.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.i.sdss.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.z.sdss.csv
CPU times: user 38.4 s, sys: 1.32 s, total: 39.7 s
Wall time: 40.8 s


**DECADE:**

Calculate zeropoints region by region...

In [15]:
%%time 

if do_calc_sdss_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.sdss.fits""" % (subdir, regionName, band)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    

                    status = DECam_tie_to_sdss(inputFile, outputFile, 
                                                    band, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.u.sdss.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.g.sdss.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.r.sdss.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.i.sdss.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.z.sdss.fits

# # # # # # # # # # # # # # #
Working on region VVDSF22
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.?.u.sdss.fits
../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.sdss.fits
../Zeropoints/zps_DECADE_FINALCUT.VVDSF22.0.g.sdss.csv
2022-01-04 19:05:42.449041
Reading in ../RawDat

2022-01-04 19:06:23.907282

Sigma-clipping...
   iter1...
2022-01-04 19:06:23.925536
2022-01-04 19:06:23.927409
2022-01-04 19:06:23.927826
  Number of rows remaining:  0
   iter2...
2022-01-04 19:06:23.930982
2022-01-04 19:06:23.932722
2022-01-04 19:06:23.933115
  Number of rows remaining:  0
   iter3...
2022-01-04 19:06:23.936053
2022-01-04 19:06:23.937815
2022-01-04 19:06:23.938050
  Number of rows remaining:  0
2022-01-04 19:06:23.939023
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 19:06:23.942199

2022-01-04 19:06:23.994932
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-E.0.z.sdss.csv output file (using pandas to_csv method)...
2022-01-04 19:06:23.995851



# # # # # # # # # # # # # # #
Working on region SN-X_err
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_err.?.u.sdss.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_e

2022-01-04 19:07:39.209995

Sigma-clipping...
   iter1...
2022-01-04 19:07:39.430218
2022-01-04 19:07:41.092895
2022-01-04 19:07:41.093728
  Number of rows remaining:  52375
   iter2...
2022-01-04 19:07:41.163513
2022-01-04 19:07:42.773132
2022-01-04 19:07:42.774049
  Number of rows remaining:  50722
   iter3...
2022-01-04 19:07:42.823269
2022-01-04 19:07:44.444426
2022-01-04 19:07:44.445145
  Number of rows remaining:  49555
2022-01-04 19:07:44.446135
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 19:07:44.462477

2022-01-04 19:07:44.684352
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.g.sdss.csv output file (using pandas to_csv method)...
2022-01-04 19:07:45.127383


../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.r.sdss.fits
../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.r.sdss.csv
2022-01-04 19:07:45.137484
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.r.sdss.fits as a pandas DataFrame...
2022-01

2022-01-04 19:08:13.777927

Sigma-clipping...
   iter1...
2022-01-04 19:08:13.798382
2022-01-04 19:08:13.800445
2022-01-04 19:08:13.800557
  Number of rows remaining:  0
   iter2...
2022-01-04 19:08:13.804109
2022-01-04 19:08:13.805675
2022-01-04 19:08:13.805952
  Number of rows remaining:  0
   iter3...
2022-01-04 19:08:13.808556
2022-01-04 19:08:13.810258
2022-01-04 19:08:13.810362
  Number of rows remaining:  0
2022-01-04 19:08:13.811189
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-04 19:08:13.814646

2022-01-04 19:08:13.875715
Writing ../Zeropoints/zps_DECADE_FINALCUT.EDFS.0.u.sdss.csv output file (using pandas to_csv method)...
2022-01-04 19:08:13.876572


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.EDFS.?.g.sdss.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.EDFS.?.r.sdss.fits
../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.EDFS.0.i.sdss.fits
../

Combine region-by-region results into a single file...

In [16]:
%%time 

if do_calc_sdss_zps: 
        
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.sdss.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.sdss.csv""" % (subdir, band)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.sdss.csv""" % (subdir, band)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band u
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.u.sdss.csv

# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.g.sdss.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.r.sdss.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.i.sdss.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.z.sdss.csv
CPU times: user 4.97 s, sys: 109 ms, total: 5.08 s
Wall time: 5.04 s
